# Classificação

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params

set_pyplot_params()

In [4]:
from utils import Or70, Pu50, Gr30

color_list3 = [Or70, Pu50, Gr30]

In [5]:
import matplotlib.pyplot as plt
from cycler import cycler

marker_cycle = cycler(marker=['s', 'o', '^'])
color_cycle = cycler(color=color_list3)
line_cycle = cycler(linestyle=['-', '--', ':'])

plt.rcParams['axes.prop_cycle'] = (color_cycle + 
                                   marker_cycle + 
                                   line_cycle)

A classificação pode ser a aplicação mais conhecida dos métodos Bayesianos, tornada famosa nos anos 90 como base da primeira geração de [spam filters](https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering).

Neste capítulo, irei demonstrar a classificação Bayesiana utilizando dados recolhidos e disponibilizados pela Dra. Kristen Gorman na Estação de Investigação Ecológica de Longo Prazo Palmer na Antárctida (ver Gorman, Williams, e Fraser, ["Ecological Sexual Dimorphism and Environmental Variability within a Community of Antarctic Penguins (Genus *Pygoscelis*)"](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0090081), Março de 2014).

Vamos utilizar estes dados para classificar os pinguins por espécie.

A célula seguinte descarrega os dados em bruto.

In [6]:
# Load the data files from 
# https://github.com/allisonhorst/palmerpenguins
# With gratitude to Allison Horst (@allison_horst)

download('https://github.com/allisonhorst/palmerpenguins/raw/master/inst/extdata/penguins_raw.csv')

## Dados sobre pinguins

Vou utilizar Pandas para carregar os dados para um "DataFrame".

In [7]:
import pandas as pd

df = pd.read_csv('penguins_raw.csv')
df.shape

O conjunto de dados contém uma linha para cada pinguim e uma coluna para cada variável.

In [8]:
df.head()

Por conveniência, vou criar uma nova coluna chamada "Espécie2" que contém uma versão mais curta dos nomes das espécies.

In [9]:
def shorten(species):
    return species.split()[0]

df['Species2'] = df['Species'].apply(shorten)

Três espécies de pinguins estão representadas no conjunto de dados:  Adélie, Chinstrap e Gentoo.

Estas espécies são mostradas nesta ilustração (por Allison Horst, disponível sob a licença [CC-BY](https://creativecommons.org/licenses/by/2.0/)):

<img width="400" src="https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/images/EaAWkZ0U4AA1CQf.jpeg" alt="Desenho de três espécies de pinguins">

As medidas que vamos utilizar são:

* Massa corporal em gramas (g).

* Flipper Comprimento em milímetros (mm).

* Culmen Comprimento em milímetros.  

* Profundidade do Culmen em milímetros.

Se não estiver familiarizado com a palavra "culmen", refere-se ao [top margin of the beak](https://en.wikipedia.org/wiki/Bird_measurement#Culmen).

O culmen é mostrado na ilustração seguinte (também por Allison Horst):

<img width="300" src="https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/images/EaAXQn8U4AAoKUj.jpeg">

Estas medições serão mais úteis para a classificação se houver diferenças substanciais entre espécies e pequenas variações dentro das espécies.  Para ver se isso é verdade, e até que ponto, vou traçar as funções de distribuição cumulativa (CDFs) de cada medida para cada espécie. 

A função seguinte toma o `DataFrame` e um nome de coluna.



In [10]:
def make_cdf_map(df, colname, by='Species2'):
    """Make a CDF for each species."""
    cdf_map = {}
    grouped = df.groupby(by)[colname]
    for species, group in grouped:
        cdf_map[species] = Cdf.from_seq(group, name=species)
    return cdf_map

A função seguinte apresenta um `Cdf` dos valores na coluna dada para cada espécie: 

In [11]:
from empiricaldist import Cdf
from utils import decorate

def plot_cdfs(df, colname, by='Species2'):
    """Make a CDF for each species.
    
    df: DataFrame
    colname: string column name
    by: string column name

    returns: dictionary from species name to Cdf
    """
    cdf_map = make_cdf_map(df, colname, by)
    
    for species, cdf in cdf_map.items():
        cdf.plot(label=species, marker='')
    
    decorate(xlabel=colname,
             ylabel='CDF')

Eis como são as distribuições para o comprimento do culmen.

In [12]:
colname = 'Culmen Length (mm)'
plot_cdfs(df, colname)

Parece que podemos usar o comprimento de culmen para identificar os pinguins Adélie, mas as distribuições para as outras duas espécies sobrepõem-se quase inteiramente.

Aqui estão as distribuições para o comprimento da barbatana.

In [13]:
colname = 'Flipper Length (mm)'
plot_cdfs(df, colname)

Utilizando o comprimento da barbatana, podemos distinguir os pinguins Gentoo das outras duas espécies.  Assim, apenas com estas duas características, parece que devemos ser capazes de classificar os pinguins com alguma precisão.

Todos estes CDFs mostram a forma sigmóide característica da distribuição normal; aproveitarei essa observação na secção seguinte.

Aqui estão as distribuições para a profundidade de culmen.

In [14]:
colname = 'Culmen Depth (mm)'
plot_cdfs(df, colname)

E aqui estão as distribuições da massa corporal.

In [15]:
colname = 'Body Mass (g)'
plot_cdfs(df, colname)

A profundidade e a massa corporal do culmen distinguem os pinguins Gentoo das outras duas espécies, mas estas características podem não acrescentar muita informação adicional, para além do que obtemos do comprimento do flipper e do comprimento do culmen.

## Modelos normais

Vamos utilizar estas características para classificar os pinguins. Vamos proceder da maneira Bayesiana habitual:

1. Definir uma distribuição prévia com as três espécies possíveis e uma probabilidade prévia para cada uma,

2. Calcular a probabilidade dos dados para cada espécie hipotética, e depois

3. Calcule a probabilidade posterior de cada hipótese.

Para calcular a probabilidade dos dados sob cada hipótese, vou utilizar os dados para estimar os parâmetros de uma distribuição normal para cada espécie.

A função seguinte toma um `DataFrame` e um nome de coluna; devolve um dicionário que mapeia a partir do nome de cada espécie para um objecto `norm`.

O `norm` é definido em SciPy; representa uma distribuição normal com uma dada média e desvio padrão.

In [16]:
from scipy.stats import norm

def make_norm_map(df, colname, by='Species2'):
    """Make a map from species to norm object."""
    norm_map = {}
    grouped = df.groupby(by)[colname]
    for species, group in grouped:
        mean = group.mean()
        std = group.std()
        norm_map[species] = norm(mean, std)
    return norm_map

Por exemplo, aqui está o dicionário de objectos "normais" para o comprimento das barbatanas de barbatanas:

In [17]:
flipper_map = make_norm_map(df, 'Flipper Length (mm)')
flipper_map.keys()

Agora suponha-se que medimos um pinguim e descobrimos que a sua barbatana é de 193 cm.  Qual é a probabilidade dessa medida sob cada hipótese?

O objecto `norm` fornece o `pdf`, que calcula a função de densidade de probabilidade (PDF) da distribuição normal.  Podemos utilizá-lo para calcular a probabilidade dos dados observados numa dada distribuição.

In [18]:
data = 193
flipper_map['Adelie'].pdf(data)

O resultado é uma densidade de probabilidade, pelo que não a podemos interpretar como uma probabilidade.  Mas é proporcional à probabilidade dos dados, pelo que podemos utilizá-la para actualizar o anterior.

Eis como calculamos a probabilidade dos dados em cada distribuição.

In [19]:
hypos = flipper_map.keys()
likelihood = [flipper_map[hypo].pdf(data) for hypo in hypos]
likelihood

Agora estamos prontos para fazer a actualização.

## A Actualização

Como sempre, utilizarei um `Pmf` para representar a distribuição prévia.  Para simplificar, vamos supor que as três espécies são igualmente prováveis.

In [20]:
from empiricaldist import Pmf

prior = Pmf(1/3, hypos)
prior

Agora podemos fazer a actualização da forma habitual.

In [21]:
posterior = prior * likelihood
posterior.normalize()
posterior

Um pinguim com uma barbatana de 193 mm é pouco provável que seja um Gentoo, mas pode ser um Adélie ou Chinstrap (assumindo que as três espécies eram igualmente prováveis antes da medição). 

A função seguinte resume os passos que acabámos de executar.

É necessário um `Pmf` representando a distribuição prévia, os dados observados, e um mapa de cada hipótese até à distribuição da característica.

In [22]:
def update_penguin(prior, data, norm_map):
    """Update hypothetical species."""
    hypos = prior.qs
    likelihood = [norm_map[hypo].pdf(data) for hypo in hypos]
    posterior = prior * likelihood
    posterior.normalize()
    return posterior

O valor de retorno é a distribuição posterior.

Aqui está novamente o exemplo anterior, utilizando `update_penguin`:

In [23]:
posterior1 = update_penguin(prior, 193, flipper_map)
posterior1

Como vimos nos CDFs, o comprimento da barbatana não distingue fortemente entre os pinguins AdÃ©lie e Chinstrap.

Mas o comprimento do culmen *can* faz esta distinção, por isso vamos usá-lo para fazer uma segunda ronda de classificação.

Primeiro estimamos as distribuições do comprimento dos cultivos para cada espécie como esta:

In [24]:
culmen_map = make_norm_map(df, 'Culmen Length (mm)')

Agora suponhamos que vemos um pinguim com culmen de 48 mm de comprimento.

Podemos utilizar estes dados para actualizar o anterior.

In [25]:
posterior2 = update_penguin(prior, 48, culmen_map)
posterior2

Um pinguim com 48 mm de comprimento de culmen tem a mesma probabilidade de ser uma Chinstrap ou Gentoo.

Utilizando uma característica de cada vez, podemos frequentemente excluir uma ou outra espécie, mas geralmente não conseguimos identificar as espécies com confiança.

Podemos fazer melhor usando múltiplas características.

## Classificação Bayesiana Ingénua

Para facilitar a realização de múltiplas actualizações, utilizarei a seguinte função, que toma um `Pmf` prévio, uma sequência de medições e uma sequência correspondente de dicionários contendo distribuições estimadas.

In [26]:
def update_naive(prior, data_seq, norm_maps):
    """Naive Bayesian classifier
    
    prior: Pmf
    data_seq: sequence of measurements
    norm_maps: sequence of maps from species to distribution
    
    returns: Pmf representing the posterior distribution
    """
    posterior = prior.copy()
    for data, norm_map in zip(data_seq, norm_maps):
        posterior = update_penguin(posterior, data, norm_map)
    return posterior

Realiza uma série de actualizações, utilizando uma variável de cada vez, e devolve o posterior `Pmf`.

Para o testar, vou usar as mesmas características que vimos na secção anterior: comprimento do culmen e comprimento do flipper.

In [27]:
colnames = ['Flipper Length (mm)', 'Culmen Length (mm)']
norm_maps = [flipper_map, culmen_map]

Agora suponhamos que encontramos um pinguim com barbatana de 193 mm de comprimento e culmen de 48.

Aqui está a actualização:

In [28]:
data_seq = 193, 48
posterior = update_naive(prior, data_seq, norm_maps)
posterior

É quase certo ser uma Chinstrap.

In [29]:
posterior.max_prob()

Podemos percorrer o conjunto de dados e classificar cada pinguim com estas duas características.

In [30]:
import numpy as np

df['Classification'] = np.nan

for i, row in df.iterrows():
    data_seq = row[colnames]
    posterior = update_naive(prior, data_seq, norm_maps)
    df.loc[i, 'Classification'] = posterior.max_prob()

Este laço adiciona uma coluna chamada `Classificação' ao `DataFrame'; contém a espécie com a máxima probabilidade posterior para cada pinguim.

Vamos então ver quantos acertámos.

In [31]:
len(df)

In [32]:
valid = df['Classification'].notna()
valid.sum()

In [33]:
same = df['Species2'] == df['Classification']
same.sum()

Existem 344 pinguins no conjunto de dados, mas dois deles não têm medidas, pelo que temos 342 casos válidos.

Destes, 324 estão classificados correctamente, o que corresponde a quase 95%.

In [34]:
same.sum() / valid.sum()

A função que se segue encapsula estas etapas.

In [35]:
def accuracy(df):
    """Compute the accuracy of classification."""
    valid = df['Classification'].notna()
    same = df['Species2'] == df['Classification']
    return same.sum() / valid.sum()

O classificador que utilizámos nesta secção é chamado "ingénuo" porque ignora as correlações entre as características.  Para ver porque é que isso importa, vou fazer um classificador menos ingénuo: um classificador que tenha em conta a distribuição conjunta das características.

## Distribuições conjuntas

Vou começar por fazer um gráfico de dispersão dos dados.

In [36]:
import matplotlib.pyplot as plt

def scatterplot(df, var1, var2):
    """Make a scatter plot."""
    grouped = df.groupby('Species2')
    for species, group in grouped:
        plt.plot(group[var1], group[var2],
                 label=species, lw=0, alpha=0.3)
    
    decorate(xlabel=var1, ylabel=var2)

Aqui está uma parcela de dispersão de comprimento de culmen e comprimento de flipper para as três espécies.

In [37]:
var1 = 'Flipper Length (mm)'
var2 = 'Culmen Length (mm)'
scatterplot(df, var1, var2)

Dentro de cada espécie, a distribuição conjunta destas medidas forma uma forma oval, pelo menos de forma aproximada.  A orientação das ovais é ao longo de uma diagonal, o que indica que existe uma correlação entre o comprimento do culmen e o comprimento do flipper.

Se ignorarmos estas correlações, estamos a assumir que as características são independentes.  Para ver o que isso parece, farei uma distribuição conjunta para cada espécie, assumindo a independência.

A função seguinte faz um discreto `Pmf` que se aproxima de uma distribuição normal.

In [38]:
def make_pmf_norm(dist, sigmas=3, n=101):
    """Make a Pmf approximation to a normal distribution."""
    mean, std = dist.mean(), dist.std()
    low = mean - sigmas * std
    high = mean + sigmas * std
    qs = np.linspace(low, high, n)
    ps = dist.pdf(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

Podemos utilizá-lo, juntamente com `make_joint`, para fazer uma distribuição conjunta do comprimento do culmen e do comprimento do flipper para cada espécie.

In [39]:
from utils import make_joint

joint_map = {}
for species in hypos:
    pmf1 = make_pmf_norm(flipper_map[species])
    pmf2 = make_pmf_norm(culmen_map[species])
    joint_map[species] = make_joint(pmf1, pmf2)

A figura seguinte compara um gráfico de dispersão dos dados com os contornos das distribuições conjuntas, assumindo a independência.

In [40]:
from utils import plot_contour

scatterplot(df, var1, var2)
for species in hypos:
    plot_contour(joint_map[species], alpha=0.5)

Os contornos de uma elipse de distribuição normal conjunta formam elipses.

Neste exemplo, como as características não estão relacionadas, as elipses estão alinhadas com os eixos.

Mas não estão bem alinhados com os dados.

Podemos fazer um modelo melhor dos dados, e utilizá-los para calcular melhores probabilidades, com uma distribuição normal multivariada.

## Distribuição Normal Multivariada

Como já vimos, uma distribuição normal univariada é caracterizada pela sua média e desvio padrão.

Uma distribuição normal multivariada é caracterizada pelos meios das características e da ** matriz decovariância**, que contém **variâncias**, que quantificam a propagação das características, e as **covariâncias**, que quantificam as relações entre elas.


Primeiro vou seleccionar as colunas que queremos.

In [41]:
features = df[[var1, var2]]

E calcular os meios.

In [42]:
mean = features.mean()
mean

Podemos também calcular a matriz de covariância:

In [43]:
cov = features.cov()
cov


Por si só, as variações e covariâncias são difíceis de interpretar.  Podemos utilizá-las para calcular desvios padrão e coeficientes de correlação, que são mais fáceis de interpretar, mas os detalhes desse cálculo não são importantes neste momento.

Em vez disso, passaremos a matriz de covariância para 'multivariada_normal', que é uma função SciPy que cria um objecto que representa uma distribuição normal multivariada.

Como argumentos é necessária uma sequência de meios e uma matriz de covariância: 

In [44]:
from scipy.stats import multivariate_normal

multinorm = multivariate_normal(mean, cov)

A função seguinte faz um objecto `multivariado_normal` para cada espécie.

In [45]:
def make_multinorm_map(df, colnames):
    """Make a map from each species to a multivariate normal."""
    multinorm_map = {}
    grouped = df.groupby('Species2')
    for species, group in grouped:
        features = group[colnames]
        mean = features.mean()
        cov = features.cov()
        multinorm_map[species] = multivariate_normal(mean, cov)
    return multinorm_map

Eis como fazemos este mapa para as duas primeiras características, comprimento da barbatana e comprimento do culminar.

In [46]:
multinorm_map = make_multinorm_map(df, [var1, var2])

## Visualizando uma Distribuição Normal Multivariada

Esta secção utiliza alguma magia NumPy para gerar gráficos de contorno para distribuições normais multivariadas.  Se isso for interessante para si, óptimo!  Caso contrário, sinta-se à vontade para saltar para os resultados.  Na próxima secção faremos a classificação real, que se revela mais fácil do que a visualização.

Começarei por fazer um mapa de contornos para a distribuição das características entre os pinguins Adélie.  

Aqui estão as distribuições univariadas para as duas características que vamos utilizar e a distribuição multivariada que acabámos de computar.

In [47]:
norm1 = flipper_map['Adelie']
norm2 = culmen_map['Adelie']
multinorm = multinorm_map['Adelie']

Vou fazer uma discreta aproximação `Pmf` para cada uma das distribuições univariadas.

In [48]:
pmf1 = make_pmf_norm(norm1)
pmf2 = make_pmf_norm(norm2)

E utilizá-los para fazer uma grelha de malha que contenha todos os pares de valores.

In [49]:
X, Y = np.meshgrid(pmf1.qs, pmf2.qs)
X.shape

A malha é representada por duas matrizes: a primeira contém as quantidades de `pmf1' ao longo do eixo `x'; a segunda contém as quantidades de `pmf2' ao longo do eixo `y'.

A fim de avaliar a distribuição multivariada para cada par de valores, temos de "empilhar" as matrizes.

In [50]:
pos = np.dstack((X, Y))
pos.shape

O resultado é uma matriz 3-D que se pode pensar como uma matriz 2-D de pares.  Quando passamos este array para `multinorm.pdf`, ele avalia a função de densidade de probabilidade da distribuição para cada par de valores.

In [51]:
densities = multinorm.pdf(pos)
densities.shape

O resultado é um conjunto de densidades de probabilidade.  Se as colocarmos num "DataFrame" e as normalizarmos, o resultado é uma aproximação discreta da distribuição conjunta das duas características.

In [52]:
from utils import normalize

joint = pd.DataFrame(densities, columns=pmf1.qs, index=pmf2.qs)
normalize(joint)

Eis como se apresenta o resultado.

In [53]:
plot_contour(joint)
decorate(xlabel=var1,
         ylabel=var2)

Os contornos de uma distribuição normal multivariada ainda são elipses, mas agora que tivemos em conta a correlação entre as características, as elipses já não estão alinhadas com os eixos.

A seguinte função encapsula os passos que acabámos de dar.

In [54]:
def make_joint(norm1, norm2, multinorm):
    """Make a joint distribution.
    
    norm1: `norm` object representing the distribution of the first feature
    norm2: `norm` object representing the distribution of the second feature
    multinorm: `multivariate_normal` object representing the joint distribution
    """
    pmf1 = make_pmf_norm(norm1)
    pmf2 = make_pmf_norm(norm2)
    X, Y = np.meshgrid(pmf1.qs, pmf2.qs)
    pos = np.dstack((X, Y))
    densities = multinorm.pdf(pos)
    joint = pd.DataFrame(densities, columns=pmf1.qs, index=pmf2.qs)
    return joint

A figura seguinte mostra um gráfico de dispersão dos dados juntamente com os contornos da distribuição normal multivariada para cada espécie.

In [55]:
scatterplot(df, var1, var2)

for species in hypos:
    norm1 = flipper_map[species]
    norm2 = culmen_map[species]
    multinorm = multinorm_map[species]
    joint = make_joint(norm1, norm2, multinorm)
    plot_contour(joint, alpha=0.5)

Como a distribuição normal multivariada tem em conta as correlações entre as características, é um modelo melhor para os dados.  E há menos sobreposições nos contornos das três distribuições, o que sugere que elas devem produzir melhores classificações.

## Um Classificador Menos Ingénuo

Numa secção anterior utilizámos o `update_penguin` para actualizar um `Pmf` anterior baseado em dados observados e uma colecção de objectos `norm` que modelam a distribuição de observações sob cada hipótese.  Aqui está novamente:

In [56]:
def update_penguin(prior, data, norm_map):
    """Update hypothetical species."""
    hypos = prior.qs
    likelihood = [norm_map[hypo].pdf(data) for hypo in hypos]
    posterior = prior * likelihood
    posterior.normalize()
    return posterior

Da última vez que utilizámos esta função, os valores em "mapa_normal" eram objectos "normais", mas também funciona se forem objectos "multivariados_normais".

Podemos utilizá-lo para classificar um pinguim com barbatana de comprimento 193 e culmen de comprimento 48:

In [57]:
data = 193, 48
update_penguin(prior, data, multinorm_map)

Um pinguim com essas medidas é quase de certeza um Chinstrap.

Agora vamos ver se este classificador faz melhor do que o classificador Bayesiano ingénuo.

Vou aplicá-lo a cada pinguim do conjunto de dados:

In [58]:
df['Classification'] = np.nan

for i, row in df.iterrows():
    data = row[colnames]
    posterior = update_penguin(prior, data, multinorm_map)
    df.loc[i, 'Classification'] = posterior.idxmax()

E calcular a precisão:

In [59]:
accuracy(df)

Acontece que é apenas um pouco melhor: a precisão é de 95,3%, em comparação com 94,7% para o classificador Bayesiano ingénuo.

## Resumo

Neste capítulo, implementámos um classificador Bayesiano ingénuo, que é "ingénuo" no sentido em que assume que as características que utiliza para a classificação são independentes.

Para ver quão má é essa suposição, implementámos também um classificador que utiliza uma distribuição normal multivariada para modelar a distribuição conjunta das características, que inclui as suas dependências.

Neste exemplo, o classificador não ingénuo é apenas marginalmente melhor.

De certa forma, isso é decepcionante.  Depois de todo esse trabalho, teria sido bom ver uma melhoria maior.

Mas de outra forma, são boas notícias.  Em geral, um classificador Bayesiano ingénuo é mais fácil de implementar e requer menos cálculos.  Se funcionar quase tão bem como um algoritmo mais complexo, poderá ser uma boa escolha para fins práticos.

Por falar em fins práticos, talvez tenha reparado que este exemplo não é muito útil.  Se quisermos identificar a espécie de um pinguim, há formas mais fáceis do que medir as suas barbatanas e o seu bico.

Mas existem *situem* utilizações científicas para este tipo de classificação.  Uma delas é o tema do trabalho de investigação com que começámos: [sexual dimorphism](https://en.wikipedia.org/wiki/Sexual_dimorphism), ou seja, diferenças de forma entre animais machos e fêmeas.

Em algumas espécies, como o peixe pescador, os machos e as fêmeas têm um aspecto muito diferente.  Em outras espécies, como os pássaros zombeteiros, são difíceis de distinguir.

E o dimorfismo vale a pena estudar porque proporciona uma visão do comportamento social, selecção sexual, e evolução. 

Uma forma de quantificar o grau de dimorfismo sexual de uma espécie é utilizar um algoritmo de classificação como o deste capítulo.  Se conseguir encontrar um conjunto de características que torne possível classificar indivíduos por sexo com elevada precisão, isso é prova de dimorfismo elevado.

Como exercício, pode utilizar o conjunto de dados deste capítulo para classificar os pinguins por sexo e ver qual das três espécies é a mais dimórfica.

## Exercícios

**Exercício:** No meu exemplo, utilizei o comprimento do culmen e o comprimento do flipper porque pareciam ser os que mais poderam distinguir as três espécies.  Mas talvez possamos fazer melhor, utilizando mais características.

Fazer um classificador Bayesiano ingénuo que utilize as quatro medidas no conjunto de dados: comprimento e profundidade do culmen, comprimento do flipper, e massa corporal.

É mais preciso do que o modelo com duas características?

In [60]:
# Solution goes here

In [61]:
# Solution goes here

In [62]:
# Solution goes here

**Exercício:** Uma das razões pelas quais o conjunto de dados do pinguim foi recolhido foi para quantificar o dimorfismo sexual em diferentes espécies de pinguins, ou seja, diferenças físicas entre pinguins machos e fêmeas.  Uma forma de quantificar o dimorfismo é utilizar medidas para classificar os pinguins por sexo.  Se uma espécie for mais dimórfica, esperamos ser capazes de os classificar com maior precisão.

Como exercício, escolher uma espécie e usar um classificador Bayesiano (ingénuo ou não) para classificar os pinguins por sexo.  Que características são mais úteis?  Que precisão se pode alcançar?

Nota: Um pinguim Gentoo tem um valor inválido para `Sexo`.  Utilizei o seguinte código para seleccionar uma espécie e filtrar os dados inválidos.

In [63]:
gentoo = (df['Species2'] == 'Gentoo')
subset = df[gentoo].copy()

In [64]:
subset['Sex'].value_counts()

In [65]:
valid = df['Sex'] != '.'
valid.sum()

In [66]:
subset = df[valid & gentoo].copy()

OK, pode terminá-lo a partir daqui.

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

In [70]:
# Solution goes here

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

In [76]:
# Solution goes here

In [77]:
# Solution goes here

In [78]:
# Solution goes here

In [79]:
# Solution goes here

In [80]:
# Solution goes here